# def explorar_tabla

Función que nos permite explora, de forma general, una tabla específica en la base de datos SQLite.

Argumentos:
*   conexion: Objeto de conexión a la base de datos.
*   nombre_tabla (str): El nombre de la tabla a explorar.

In [ ]:
def explorar_tabla(conexion, nombre_tabla):
    cursor = conexion.cursor()
    print("------------")
    print(f"\nExplorando la tabla '{nombre_tabla}'")
    try:
        # Ver el esquema de la tabla
        cursor.execute(f"PRAGMA table_info('{nombre_tabla}');")
        columnas = cursor.fetchall()
        print(f"\nEsquema de la tabla '{nombre_tabla}':")
        for columna in columnas:
            print(f"  - Nombre: {columna[1]}, Tipo: {columna[2]}, ¿Nulo?: {columna[3]}, Clave Primaria: {columna[5]}")

        # Mostrar las primeras 10 filas
        cursor.execute(f"SELECT * FROM {nombre_tabla} LIMIT 10;")
        primeras_filas = cursor.fetchall()
        if primeras_filas:
            print(f"\nPrimeras 10 filas de la tabla '{nombre_tabla}':")
            for fila in primeras_filas:
                print(fila)
        else:
            print(f"\nLa tabla '{nombre_tabla}' está vacía.")

        # Contar el número total de filas
        cursor.execute(f"SELECT COUNT(*) FROM {nombre_tabla};")
        num_filas = cursor.fetchone()[0]
        print(f"\nNúmero total de filas en '{nombre_tabla}': {num_filas}")

    except sql.Error as e:
        print(f"Error al explorar la tabla '{nombre_tabla}': {e}")